<span style="font-size:8pt">*ÉTS Montréal, GPA671 : Introduction à l’intelligence artificielle. Date : le 19/08/21. Auteur : Jérôme Rony. Version : 1.0*</span>

# Laboratoire 3 : Regroupement et réduction de dimensionalité

## Exercice 1 : K-Moyennes

### Rappels de cours

L'algorithme des K-Moyennes permet de trouver une partition dure des données en K groupes. Le but de l'algorithme est de minimiser la quantité :
\begin{equation}
J = \sum_{i=1}^N\sum_{k=1}^K r_{ik}||{\bf x}_i - {\bf \mu}_k||^2
\end{equation}
où ${\bf x}_i$ est une donnée, ${\bf \mu}_k$ est le centre du groupe $k$ et $r_{ik}\in\{0, 1\}$ est un indicateur binaire qui indique à quel groupe $k$ est associée la donnée ${\bf x}_i$. Plus formellement, on peut définir cet indicateur de la manière suivante :
\begin{equation}
r_{ik} = \begin{cases}1 &\text{si}\, k = \mathrm{arg\,min}_j||{\bf x}_i - {\bf \mu}_j||^2 \\0 &\text{sinon.}\end{cases}
\end{equation}
Pour chaque exemple, on calcule donc la distance à chaque centre et on cherche l'indice du centre le plus proche.

Une fois que cette association est trouvée, on met à jour les centres ${\bf \mu}_k$ grâce à l'équation suivante:
\begin{equation}
{\bf \mu}_k = \frac{\sum_{i=1}^N r_{ik}{\bf x}_i}{\sum_{i=1}^N r_{ik}}
\end{equation}
ce qui correspond à une moyenne sur l'ensemble des données associées au centre $k$.

Enfin, on répète cette procédure jusqu'à ce que la quantité $J$ ne change plus. 

### Questions

Dans un premier temps, nous allons utiliser des données synthétiques pour observer le comportement de l'algorithme et vérifier que l'implémentation est correcte. Ensuite nous utiliserons l'algorithme des K-moyennes pour compresser une image en réduisant le nombre de couleurs utilisées pour la représenter.

1. Charger des données synthétiques à l'aide du code suivant et les afficher à l'aide de la fonction `scatter` de Matploltib avec une couleur différente pour chaque classe.
```python
from sklearn import datasets
X, y = datasets.make_blobs(n_samples=[100, 100, 1000], cluster_std=[1.0, 2.5, 0.5], random_state=17)
```
2. La première étape de l'algorithme des K-Moyennes est d'initialiser les centres ${\bf \mu}_k$.
    1. Une stratégie simple pour les initialiser est de choisir aléatoirement les centres. Implémenter une **fonction** `random_init` permettant de choisir aléatoirement $K$ données parmi les toutes les données `X`. Afficher 3 centres choisis lors de 5 initialisations indépendantes avec des marqueurs différents parmi `X`. Commenter cette initialisation.    
    2. La stratégie la plus commune est appelée *k-means++* et correspond à choisir les centres un-par-un aléatoirement, avec un biais vers les points les plus éloignées des centres déjà choisis. Une fois le premier centre choisi aléatoirement, on calcule la distance entre les données `X` et le centre le plus proche. Le centre suivant est choisi aléatoirement parmi les points restant avec une probabilité proportionnelle à cette distance. On recommence ce choix jusqu'à avoir $K$ centres. Implémenter cette stratégie dans une **fonction** `kmeans_plusplus` et afficher les centres choisis lors de 5 initialisations indépendantes. Commenter cette initialisation.
    > Utiliser la fonction `numpy.random.choice` qui permet de spécifier une probabilité `p` de choisir chaque élément d'un tableau `a`.
3. Implémenter l'algorithme des K-Moyennes permettant d’utiliser n’importe quel $K$ et de choisir l'initialisation (aléatoire ou *k-means++*) dans une classe qui aura les méthodes suivantes :
    * `fit` qui prend en entrées des données `X` et applique l'algorithme des K-moyennes avec l'initialisation choisie.
    * `predict` qui prend en entrée des données `X` et renvoie le groupe prédit pour chaque échantillon.
4. Tester cet algorithme sur les données générées synthétiquement avec les deux initialisations et observer la valeur de $J$ lors des itérations de l'algorithme sur un graphique avec une échelle logarithmique pour l'axe des ordonnées. Afficher les centres obtenus ainsi que les groupes prédits pour les données pour chaque initialisationn. Commentez l'initialisation aléatoire.
5. Il est courant de recommencer l'algorithme des K-moyennes plusieurs fois pour obtenir une meilleure solution, c'est-à-dire avec une valeur de $J$ plus basse. Ajouter une méthode `fit_restart` à la classe précédente qui applique la méthode `fit` plusieurs fois (contrôlé par un paramètre `n_restart`) et garde le meilleur résultat. Comparer les résultats obtenus entre `fit_restart` avec `n_restart=5` et `fit` en utilisant l'initialisation *k-means++*.
6. Charger l'image `chelsea` depuis le paquet `skimage.data` et l'afficher. Appliquer l'algorithme de K-moyennes sur cette image avec différentes valeurs de K et afficher les images compressées pour les valeurs choisies. Pour cela il faut trouver les centres correspondant à chaque pixel de l'image puis créer une nouvelle image dont les valeurs des pixels sont celles de leur centre correspondant.
> Les images sont généralement stockées en uint8 (8 bits non signé) sur la plage d'entiers $[0, 255]$. Il faut donc commencer par les convertir en float32 et les ramener dans $[0, 1]$ ce qui peut être fait comme suit: `chelsea = data.chelsea().astype(np.float32) / 255`.

7. Recommencer avec les images `astronaut` et `coffee` de `skimage.data`. Commenter les résultats. Dans quels cas cet algorithme n'est pas adapté pour compresser des images?

## Exercice 2 : Réduction de dimensionnalité avec PCA

### Rappels de cours

L'analyse en composantes principales permet de réduire la dimensionnalité d'un problème en choisissant des directions orthogonales selon lesquelles les données ont la variance la plus élevée dans l'espace d'entrée.

Afin de calculer ces directions, la première étape est de centrer les données $X\in\mathbb{R}^{n\times d}$ :
\begin{equation}
\tilde{X} = X - \bar{X} \qquad \bar{X} = \frac{1}{n}\sum_{i=1}^n X_i
\end{equation}
Ensuite, on calcule la matrice de covariance $C\in\mathbb{R}^{d\times d}$ des composantes de X:
\begin{equation}
C = \mathrm{cov}(\tilde{X}, \tilde{X}) = \tilde{X}^\top \tilde{X}
\end{equation}
Une fois la matrice de covariance obtenue, on cherche les valeurs propres et vecteurs propres de $C$, c’est-à-dire les valeurs $\lambda \in \mathbb{R}$ et vecteurs ${\bf \varphi}\in\mathbb{R}^{d}$ tels que $C{\bf \varphi}=\lambda {\bf \varphi}$.

On peut alors projeter les données centrées dans un espace de dimension réduite (ici $p$):
\begin{equation}
Z = \tilde{X} \Phi \qquad \Phi = [\varphi_1, \varphi_2, \dots, \varphi_p] \in \mathbb{R}^{d\times p}
\end{equation}

On peut enfin reconstruire les données originales comme suit:
\begin{equation}
\hat{X} = Z\Phi^\top + \bar{X}
\end{equation}

### Questions

Dans cet exercice, on cherche à trouver des groupes d'images dans une banque d'images. Plus spécifiquement, nous allons utiliser le jeu de données de visages *Olivetti* qui contient 10 photos des visages de 40 personnes, soit 400 images au total.

1. Charger le jeu de données *Olivetti* à l'aide de `sklearn.datasets.fetch_olivetti_faces`. Afficher 5 images choisies aléatoirement. Séparer les données en une base d'entrainement et une base de test avec une proportion de 40% pour l'entrainement et 60% pour le test.
> Attention à bien utiliser l'option `stratify` de `sklearn.model_selection.train_test_split` pour assurer que toutes les étiquettes se retrouvent dans l'entrainement et le test.
2. Entraîner un classifieur naïf Bayésien à l'aide de `sklearn.naive_bayes.GaussianNB` sur les données d'entraînement brutes et calculer le taux d'exactitude sur les données de test.
3. Calculer les valeurs propres et vecteurs propres de la matrice de covariance des données d'entraînement. Afficher la somme cumulative normalisée des valeurs propres ordonnées par ordre décroissant. À quoi correspond cette somme cumulée?
> La matrice de covariance étant symmétrique, on peut utilise `numpy.linalg.eigh` qui est plus rapide que `numpy.linalg.eig`.
4. Afficher les 10 premiers vecteurs propres sous forme d'image (remarquer la dimension identique aux images originales). Que représentent ces images?
5. Projeter les données d'entrainement sur les 5 premiers vecteurs propres correspondant aux 5 plus grandes valeurs propres. Reconstruire 8 images aléatoirement choisies et les afficher en dessous des 8 images originales correspondantes. Que se passe-t-il lorsqu'on projette les données sur un plus grand nombre de vecteurs propres? Plus petit?
6. Entrainer un classifieur naïf Bayésien sur les données projetées sur les 10 premiers vecteurs propres et évaluer sa performance sur les données de test.
7. Étudier la performance du classifieur naïf Bayésien en fonction du nombre de valeurs propres utilisées pour la projection. Mesurer le taux d'exactitude ainsi que les temps d'entrainement d'inférence. Conclure sur le choix du nombre de vecteurs propres à utiliser lors de la projection.
8. Utiliser la méthode `sklearn.discriminant_analysis.LinearDiscriminantAnalysis` qui effectue une réduction de dimensionnalité **et** entraine un classifieur naïf Bayésien. Comparer le taux d'exactitude obtenu ainsi que le temps d'inférence avec la solution précédemment choisie. Expliquer.